# What's happening here?

4_link_video_behavior.ipynb

Run after 1_, 2_, and 3_

Made in environment **PMS_Study**

## TO DO: bekijk file PMS_analysis/2_preparation

Probeer veel met loops en functions te werken om de code overzichtelijk te houden

 * Start with file cleanData.csv (geeft informatie over van welke participant welke files bestaan)
 * Load timings.csv file that Tilia and Sofie made
 * Start loop from the top and use Order and A-B
 * In loop check if file (corresponding CSV) exists in Directory (/10001 or /10002 depedent of A/B): later refered to as behaviordata
 * In loop check if file (corresponding .WEBM) exists in Directory
 
If both exist:
- Check if output of Openface exists in Z:\shares\ghepmk_data\2020_Kappen_PMS\10002\video_features (or /10001 dependent of A/B)
- IF True:
Load this as facedata
- IF False:
Print: "Video file present for file (insert filename) but no openface present)"
Break/skip

Use filename/pptnum to search for right starting time in timings.csv variable
- Find timing for beep.wav in behaviordata. 
- Set new zero point (link between the timings.csv point and this)
- Make mean scores on a few variables (we will expand this later) for the corresponding frames of each of the trials.
- To do this, you should write a function that takes the right points of a trial (start, end, trialnum etc) and save this data to a dataframe

Compute this in a way that you would be able to add it to allPMSdata.cvs

Add it all to a new dataframe

Write to csv

In [1]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math
import time

from tqdm import tqdm
from pathlib import Path

import re

In [55]:
# Create all relevant directories
data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')
date_dir = "06102021/"
data_all_dir = os.path.join(data_dir, date_dir)

# Load base data
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")
allData = pd.read_csv(data_all_dir + "allPMSdata.csv", encoding = "ISO-8859-1")
timings = pd.read_csv(data_all_dir + "timings.csv", encoding = "ISO-8859-1")
timings = timings.rename(columns={"ï»¿ImageGroup": "ImageGroup"}, index={'ONE': 'Row_1'}) # Rename first column, because was formatted weird





In [56]:
timings

,ImageGroup,Date,pptnumber,StartTime,EndTime,OtherComments,Unnamed: 6,Exclusion,TestMoment
0,B,8/27/2020,14,0.26,?,"looks to the side, tv playing",NaN,1,0
1,B,9/28/2020,66,0.16,5.38,looks at phone. Same ppt as 14,NaN,1,0
2,B,8/31/2020,106,0.36,?,NaN,NaN,0,2
3,B,9/23/2020,119,1.14,6.47,drinking tea,NaN,0,2
4,B,7/9/2021,123,0.31,5.14,typing (texting?),NaN,0,2
...,...,...,...,...,...,...,...,...,...
520,A,8/31/2021,674,1.44,7.21,"in background, there's a picture of a face (no...",NaN,0,2
521,A,7/8/2021,676,2.08,7.05,hair bit over left eye,NaN,0,1
522,A,9/24/2021,678,?,?,no beeps,NaN,0,2
523,A,9/30/2021,679,0.12,5.11,"eyebrow piercing, chin not visible",NaN,0,2


In [54]:
timings["ï»¿ImageGroup"]

0      B
1      B
2      B
3      B
4      B
      ..
520    A
521    A
522    A
523    A
524    A
Name: ï»¿ImageGroup, Length: 525, dtype: object

In [41]:
# Get all OpenFace files
os.chdir(data_dir)
AFiles = glob.glob("10001/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
BFiles = glob.glob("10002/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory



In [67]:
# BFiles

In [78]:
openface

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,0.063975,-0.002084,-0.997949,0.072608,-0.148678,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,0,0.001,0.98,1,0.035288,-0.006892,-0.999353,0.055273,-0.158703,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,0,0.002,0.98,1,0.024994,0.009102,-0.999646,0.052177,-0.153878,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,0,0.003,0.98,1,0.033676,-0.017344,-0.999282,0.039422,-0.162667,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,5,0,0.004,0.98,1,-0.011143,0.000171,-0.999938,-0.005280,-0.147903,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10423,10424,0,10.423,0.98,1,0.042126,-0.121335,-0.991717,-0.178407,-0.103466,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
10424,10425,0,10.424,0.98,1,0.042254,-0.120352,-0.991832,-0.178183,-0.102205,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
10425,10426,0,10.425,0.98,1,0.042256,-0.120901,-0.991765,-0.177429,-0.103008,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
10426,10427,0,10.426,0.88,1,0.042502,-0.117136,-0.992206,-0.176501,-0.099713,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [77]:


index = 0
for pptnum in tqdm(timings["pptnumber"]): # Loop over all ppt's that had a video available (therefore present in timings.csv)
    
    print("processing pptnum: " + str(pptnum))
    idx = np.where(allData['ID'] == int(pptnum))[0]
    if timings["Exclusion"][index] == 1: # Only continue the loop if there is no Exclusion marked
        print("Exclusion file for ppt: " + str(pptnum) + " at moment: " + timings["ImageGroup"][index])
    else:
        if len(idx) < 1: # Check if occurences in datafile exist for this participant/video file
            print("No input data found for video file of pptnum: " + str(pptnum) + " with video file " + timings["ImageGroup"][index] )
        else:           
            moment = timings["ImageGroup"][index]
            if moment ==  'A':
                indices = [i for i, s in enumerate(AFiles) if str(pptnum) in s]
                fileDir = AFiles[indices[0]] 
            elif moment == 'B':
                indices = [i for i, s in enumerate(BFiles) if str(pptnum) in s]
                fileDir = BFiles[indices[0]] 
            else:
                print("Some error with testing moment")
                break
            
            if len(indices) > 1:
                print("Too many OpenFace files for ppt: " + str(pptnum) + " at moment " + moment)
            else:
                openface = pd.read_csv(fileDir, encoding = "ISO-8859-1")
            
            
            
            
    index += 1 # Create counter
    if index > 10:
        break

processing pptnum: 14
Exclusion file for ppt: 14 at moment: B
processing pptnum: 66
Exclusion file for ppt: 66 at moment: B
processing pptnum: 106
No input data found for video file of pptnum: 106 with video file B
processing pptnum: 119
No input data found for video file of pptnum: 119 with video file B
processing pptnum: 123
No input data found for video file of pptnum: 123 with video file B
processing pptnum: 124
No input data found for video file of pptnum: 124 with video file B
processing pptnum: 125
No input data found for video file of pptnum: 125 with video file B
processing pptnum: 126
No input data found for video file of pptnum: 126 with video file B
processing pptnum: 127
No input data found for video file of pptnum: 127 with video file B
processing pptnum: 129
processing pptnum: 131


In [9]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [21]:
# listOfPositions = getIndexes(allData, pptnum)
# print(listOfPositions)

pptnum

14

In [24]:
indx = np.where(allData['ID'] == int(pptnum))[0]
print(indx)
len(indx)

[]


0